In [1]:
from imutils.paths import list_files
import pandas as pd
import os, re

In [2]:
ROOT_PATH    = '/home/jovyan/project/MidnightTable'
DATASET_PATH = f'{ROOT_PATH}/TableSeekTag/src/data/zomato' 

- 메뉴 리스트가 없는 식당에서는 It's empty here!\n 이라는 문구가 있어  
  해당 문구가 있는 식당 열을 csv 파일에서도 제거하였습니다.

In [3]:
## 멋져보이고 싶어서 만든 굳이 함수로 안 만들었어도 되는 함수들..
remove_ent    = lambda text: text.replace('\n', '')
read_text     = lambda text_file: open(text_file, 'r').readlines() 
length_filter = lambda text, min = 5, max = 70: all([len(text) > min, len(text) <= max])

In [4]:
text_files = sorted(list_files(f'{DATASET_PATH}/scraping_datas/'))
csv        = pd.read_csv(f'{DATASET_PATH}/HyderabadResturants.csv')

## 필요한 정보가 식당 이름, 요리 카테고리여서 원본 DataFrame에서 names, cuisine 칼럼만 남김
csv        = csv[['names', 'cuisine']]
len(csv)

657

In [5]:
int_pattern     = re.compile('₹[0-9]*')
float_pattern   = re.compile('₹[0-9]*.[0-9]*')

code_pattern    = re.compile('use code [a-zA-Z]*')
s_bracket_pattern = re.compile('\[[^]]*\]')
bracket_pattern = re.compile('\([^)]*\)')

kg_pattern   = re.compile('[0-9]* kg')
gram_pattern = re.compile('[0-9]* gm')

In [6]:
## 텍스트 파일들 불러오기
texts        = [read_text(text_file) for text_file in text_files]

## 메뉴 정보가 없는 식당 정보 인덱스 저장용
no_menu_list, restaurants = [], []

filters = (
            'veg only', 'recommended', 
            'use code tasty', 'newly launched',
            "today's exclusive dishes",
            'bestseller', 'not eligible for coupons',
            'download the app', "[chef's special]"
        )

In [7]:
for odx, text in enumerate(texts, 1): 
    ## 메뉴가 없는 녀석들 제거용
    if "It's empty here!\n" in text: 
        filter_char = "It's empty here!\n"
        no_menu_list.append(odx)
    
    total_menus = []
    # try:
    ## 가격정보 (₹)가 담겨있는 녀석들의 인덱스 탐색
    prices = []
    for idx, t in enumerate(text):
        condition = [
                        float_pattern.fullmatch(remove_ent(t)) != None,
                        int_pattern.fullmatch(remove_ent(t)) != None
                    ]
        if any(condition): prices.append(idx)
            
    if not odx in no_menu_list:
        ## 가격 정보가 있는 첫 인덱스 (가격정보로 부터 2 줄 앞에 메뉴정보가 있는데, 넉넉히 5줄 앞부터로 시작)
        ## 부터 마지막 가격정보 인덱스까지만 가져옴.
        try: 
            menus = text[prices[0] - 5: prices[-1] + 1]
            ## 5개 중에 메뉴 하나 정도는 있겠지 탐색법,,
            for idx in range(0, len(menus), 5):
                menu = menus[idx: idx + 5]

                ## 필요없는 데이터 필터링
                menu = [m.lower() for m in menu if not m.isupper()]

                menu = [m.lower() for m in menu if length_filter(m, max = 40)]
                menu = [m for m in menu if not '₹' in m]
                menu = [remove_ent(m) for m in menu if not 'votes' in m]
                menu = [m for m in menu if bracket_pattern.fullmatch(m) == None]
                menu = [m for m in menu if code_pattern.fullmatch(m) == None]
                menu = [m for m in menu if m not in filters]

                menu = [bracket_pattern.sub('', m) for m in menu]
                menu = [s_bracket_pattern.sub('', m) for m in menu]
                menu = [kg_pattern.sub('', m) for m in menu]
                menu = [gram_pattern.sub('', m) for m in menu]

                total_menus += menu
            restaurants.append(set(total_menus))
        
        ## 가격정보가 없는 녀석들,,,
        except: no_menu_list.append(odx)
    else: pass

In [8]:
## csv 파일에서 메뉴 정보가 없는 식당 정보들 삭제
csv = csv.drop(no_menu_list, axis = 0)
len(csv)

620

In [9]:
csv['menus'] = restaurants
csv.head()

,names,cuisine,menus
0,Sahara Bakers,"Chinese, Bakery, Sichuan, Pizza, Burger","{chicken capsicum pizza, paneer chilly, red ve..."
1,KFC,"Burger, Fast Food, Biryani, Desserts, Beverages","{hot & crispy chicken -6pc, 6pc hot & crispy w..."
2,Subbaiah Gari Hotel,"South Indian, Andhra, Mithai","{pongal, allam pickle , thumsup 750ml, spl hal..."
3,Paradise Biryani,"Biryani, Kebab, Desserts, Beverages","{kiwi blast mocktail, chocolate cake , mango s..."
5,Shah Ghouse Hotel & Restaurant,"North Indian, Chinese, Mughlai, Mandi, Sichuan...","{zabaan, irani tea kettle , bheja masala, tand..."


In [10]:
csv.to_csv(f'../../TableSeekTag-private-/data/zomato.csv')